In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


df = pd.read_csv('../data/processed/merged_analysis.csv')

#  datetime and ordinal convertion
for col in ['Week', 'Date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)
    df[f"{col}_ordinal"] = df[col].map(lambda d: d.toordinal())
    df.drop(col, axis=1, inplace=True)

# common features
features = ['Week_ordinal', 'Date_ordinal']

# ===== USD_TRY Model =====
X_usd = df[features].copy()
y_usd = df['USD_TRY'].copy()

# remove NaN
mask_usd = y_usd.notna()
X_usd = X_usd[mask_usd]
y_usd = y_usd[mask_usd]

# Train Test Split
X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(
    X_usd, y_usd, test_size=0.2, random_state=42
)

# Scaler
scaler_u = StandardScaler()
X_train_u = scaler_u.fit_transform(X_train_u)
X_test_u  = scaler_u.transform(X_test_u)

# Models
models_usd = {
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
}

print("=== Result of Guessing USD Prices ===")
for name, model in models_usd.items():
    model.fit(X_train_u, y_train_u)
    print(f"{name}: R^2 = {model.score(X_test_u, y_test_u):.3f}")

# ===== FİYAT Model =====
X_fiyat = df[features].copy()
y_fiyat = df['FİYAT'].copy()

# Remove Nan
mask_fiyat = y_fiyat.notna()
X_fiyat = X_fiyat[mask_fiyat]
y_fiyat = y_fiyat[mask_fiyat]

#Trains Test Split
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_fiyat, y_fiyat, test_size=0.2, random_state=42
)

# Scaler
scaler_f = StandardScaler()
X_train_f = scaler_f.fit_transform(X_train_f)
X_test_f  = scaler_f.transform(X_test_f)

# Models
models_fiyat = {
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
}

print("\n=== Result of Guessing Gold Prices ===")
for name, model in models_fiyat.items():
    model.fit(X_train_f, y_train_f)
    print(f"{name}: R^2 = {model.score(X_test_f, y_test_f):.3f}")


=== Result of Guessing USD Prices ===
KNN: R^2 = 0.996
DecisionTree: R^2 = 0.998
RandomForest: R^2 = 0.999

=== Result of Guessing Gold Prices ===
KNN: R^2 = 0.988
DecisionTree: R^2 = 0.977
RandomForest: R^2 = 0.992


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


df = pd.read_csv('../data/processed/merged_analysis.csv')

#) convertion of datetime and ordinal
for col in ['Week', 'Date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)
    df[f'{col}_ordinal'] = df[col].map(lambda d: d.toordinal())
    df.drop(col, axis=1, inplace=True)

# features
features = ['Week_ordinal', 'Date_ordinal']
X = df[features]
y = df['USD_TRY']

# 4) remove NaN
mask = y.notna()
X = X.loc[mask]
y = y.loc[mask]

# 5) Train Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 6) Scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# 7) Model and prediction
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# 8) Metrics
mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f'R²   : {r2:.3f}')
print(f'MSE  : {mse:.3f}')
print(f'RMSE : {rmse:.3f}')
print(f'MAE  : {mae:.3f}')

# 9) Comprasion of Random 10 Values
comp = (pd.DataFrame({'Real Value of USD': y_test.values,
                      '      Prediction Value of USD': y_pred})
         .sample(10, random_state=42))
print("\nRandom 10 Value:\n", comp)


R²   : 0.999
MSE  : 0.095
RMSE : 0.309
MAE  : 0.156

Random 10 Value:
     Real Value of USD        Prediction Value of USD
61           7.943958                       7.686385
57           7.300596                       7.101137
0            8.610068                       8.578679
43          17.257456                      16.964853
5           18.623406                      18.631864
36          22.215686                      21.055235
16           5.791602                       5.855109
12          26.939160                      26.654791
25           8.108340                       8.161888
60           9.325512                       9.229617


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = pd.read_csv('../data/processed/merged_analysis.csv')

# 2) convert datetime and ordinal
for col in ['Week', 'Date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)
    df[f'{col}_ordinal'] = df[col].map(lambda d: d.toordinal())
    df.drop(col, axis=1, inplace=True)

# 3) Features and Target
features = ['Week_ordinal', 'Date_ordinal']
X = df[features]
y = df['FİYAT']

# 4) remove NaN
mask = y.notna()
X = X.loc[mask]
y = y.loc[mask]

# 5) Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# 6) Scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# 7) Precit and Model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

# 8) Metrics
mse  = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f'R²   : {r2:.3f}')
print(f'MSE  : {mse:.3f}')
print(f'RMSE : {rmse:.3f}')
print(f'MAE  : {mae:.3f}')

# 9) Comparison
comparison = (
    pd.DataFrame({
        'Real Value': y_test.values,
        'Prediciton Value': y_pred
    })
    .sample(10, random_state=42)
)
print("\nRandom 10 Value :")
print(comparison)


R²   : 0.992
MSE  : 3486.367
RMSE : 59.045
MAE  : 31.180

Random 10 Value :
    Real Value  Prediciton Value
9       297.91          303.6371
11      280.10          276.2929
0       478.84          473.7415
13      259.50          258.1561
5       413.81          426.1252
8      1091.54         1077.7337
2      1201.91         1188.5121
1       225.32          229.2215
14     1689.14         1567.2401
4      1710.53         1768.0335
